#       X Corp International: 
# Multi-Family Development Proposal
##             (Texas)

## The Business Problem:

* X Corp International has engaged your firm to assist them in determining the next best location to develop a large multi-family apartment complex.
* They have asked that you provide the top 5 locations in Texas for this project and would like to know the average income, as well as how many people 
* are expected to be moving to the area. Your job is to collate, prepare and analyze the data through any means necessary to provide a reccomendation 
* in a visual presentation for X Corp Intl. leaderhip.

In [1]:
# Import the potential necessary packages 

import numpy as np                        # Library to handle data in a vectorized manner
import pandas as pd                       # For dataframe manipulation
import matplotlib as mpl
import matplotlib.pyplot as plt
import json                               # For reading in JSON files
import sklearn as skl                     # For machine learning and cluster analysis
import folium                             # Renders map visuals
import matplotlib.cm as cm
import matplotlib.colors as colors        # Geospatial map colors
from sklearn.cluster import KMeans        # For cluster analysis
import requests                           # Library to handle requests
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim     # Convert an address into latitude and longitude values

print("Library Import Successful")

Library Import Successful


 * Insert Credentials to request data from the FourSquare API ** Disclaimer** This information will not be present on the GitHub page, however you can create your own credentials from this link https://developer.foursquare.com/

In [2]:
CLIENT_ID     =  "VJN5PS0BZKSMGVYOTPF3QBB2SCYUJAXNBJ1U50JSRO434XQQ"
CLIENT_SECRET =  "3VNQ4YYP2BZYZVMU234B0VWRNHXSRVQA4IVGVBLHQ4SKRGYR"
VERSION       =  "20200730"              # This is todays date   
LIMIT         =  100                     # Limit the number of locations within your radius
RADIUS        =  500                     # Define the radius

print("Your Credentials:")
print("Client ID: " + CLIENT_ID)
print("Client Secret: " + CLIENT_SECRET)

Your Credentials:
Client ID: VJN5PS0BZKSMGVYOTPF3QBB2SCYUJAXNBJ1U50JSRO434XQQ
Client Secret: 3VNQ4YYP2BZYZVMU234B0VWRNHXSRVQA4IVGVBLHQ4SKRGYR


In [36]:
df = pd.read_excel("C:/Users/thech/Desktop/latlong.xlsx", 
                   header = 0, delim_whitespace = True
                  )
df

df2 = pd.read_excel("C:/Users/thech/Desktop/Texas_Income.xlsx", 
                    header = 0, delim_whitespace = True
                   )
df2

,Zip,Median household income,Margin of Error,Mean income,Margin of Error.1,Per capita income,Margin of Error.2,"Percent of Households with incomes $200,000 or more",Margin of Error.3,Householder 25 to 44 years,Margin of Error.4,Householder 45 to 64 years,Margin of Error.5,Householder 65 years and over,Margin of Error.6
0,73430,61848,8782,65312,7881,24994,3081,0.011,0.020,75907.0,5277,75969.0,8629,68516.0,21237
1,73931,57143,10036,66882,10614,30638,5643,0.016,0.021,101378.0,5706,112437.0,5479,68322.0,11643
2,73949,46023,8445,57662,7639,24235,3239,0.019,0.014,60681.0,3264,73269.0,3916,51160.0,5786
3,75001,73094,3923,88172,4600,50016,3077,0.066,0.017,83324.0,3920,99116.0,4997,56875.0,4410
4,75002,101661,3855,127734,5901,41474,2145,0.141,0.014,95143.0,14124,113084.0,9849,63517.0,16568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,88260,53444,4858,69958,5945,22786,1926,0.032,0.019,38613.0,3090,41250.0,27819,27257.0,8030
1942,88263,103571,85982,100292,38132,42421,16166,0.149,0.215,-1.0,.,-1.0,.,-1.0,.
1943,88410,-1,.,-1,N,13077,4251,0.000,0.647,NaN,NaN,NaN,NaN,NaN,NaN
1944,88415,37576,6282,60581,14241,24065,4939,0.021,0.018,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df3 = pd.merge(df,
               df2,
               left_on  = "Zip", 
               right_on = "Zip",
               how = "inner"
              )
df3

,Zip,City,Latitude,Longitude,Median household income,Margin of Error,Mean income,Margin of Error.1,Per capita income,Margin of Error.2,"Percent of Households with incomes $200,000 or more",Margin of Error.3,Householder 25 to 44 years,Margin of Error.4,Householder 45 to 64 years,Margin of Error.5,Householder 65 years and over,Margin of Error.6
0,75087,Rockwall,32.933820,-96.454460,96789,5092,110964,5200,39436,1992,0.106,0.018,40746.0,4066,43839.0,11117,38861.0,5332
1,78879,Rio Frio,29.628266,-99.738470,-1,.,54862,32785,32527,14528,0.000,0.421,-1.0,.,71875.0,69672,50054.0,19169
2,79789,Wink,31.753101,-103.157370,71094,25447,81702,14174,26936,4134,0.039,0.031,-1.0,.,-1.0,.,-1.0,.
3,79836,Clint,31.570185,-106.213300,38668,2910,46888,4455,14603,1356,0.008,0.006,25703.0,19571,26154.0,13086,31471.0,22720
4,78108,Cibolo,29.574127,-98.233080,94670,4656,104663,3836,33839,1316,0.074,0.013,99688.0,31548,67727.0,24577,63309.0,18542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,75779,Neches,31.794191,-95.661964,51250,35593,59218,27729,19130,5822,0.000,0.564,40431.0,12004,44674.0,18593,-1.0,.
1291,76828,Burkett,32.023828,-99.235970,75156,58059,72541,16058,29916,6117,0.000,0.364,37255.0,18546,45461.0,6549,41689.0,12284
1292,79062,Morse,36.004239,-101.546720,68125,41757,98180,39461,36854,17722,0.055,0.084,53218.0,6745,60379.0,9440,35188.0,4214
1293,76957,Wall,31.359844,-100.206705,-1,.,-1,**,-1,.,0.000,0.000,94359.0,10094,31250.0,8754,43400.0,13979


* Check for Any Null values and drop them if necessary

In [43]:
df.isnull().values.any()

False

* Use the Geopy library to obtain the coordinates for the city of Houston
* Render a map of the city using the Folium package

In [44]:
address = "TX"

geolocator = Nominatim(user_agent = "Coursera_Capstone")
location   = geolocator.geocode(address)

print(location.latitude, location.longitude)

31.8160381 -99.5120986


In [15]:
# Create map of Texas using the above latitude and longitude values
map_TX = folium.Map(location = [location.latitude, location.longitude], zoom_start = 6)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng]    ,
        radius = 4    ,
        popup = label ,
        color = 'navy',
        fill  = True  ,
        fill_color = 'mediumblue',
        fill_opacity = .6,
        parse_html = False).add_to(map_TX)  
    
map_TX